# Lab | Data cleaning and wrangling
For this lab, we will be using the same dataset we used in the previous labs. We recommend using the same notebook since you will be reusing the same variables you previous created and used in labs.

## Instructions
So far we have worked on EDA. This lab will focus on data cleaning and wrangling from everything we noticed before.

1. We will start with removing outliers. So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.
2. Create a copy of the dataframe for the data wrangling.
3. Normalize the continuous variables. You can use any one method you want.
4. Encode the categorical variables
5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.
6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

### Hint for Categorical Variables
You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):
* One hot to state
* Ordinal to coverage
* Ordinal to employmentstatus
* Ordinal to location code
* One hot to marital status
* One hot to policy type
* One hot to policy
* One hot to renew offercustomer_df
* One hot to sales channel
* One hot vehicle class
* Ordinal vehicle size

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
* given that column "coverage" in the dataframe "data" has three categories:
* "basic", "extended", and "premium" and values are to be represented in the same order.

In [1]:
# Importing the libraries.
import pandas as pd        # Data processing, CSV file I/O (e.g. pd.read_csv)
import numpy as np         # Linear algebra
import seaborn as sns      # For data visualization
import re
import matplotlib.pyplot as plt      # For data visualization purposes
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, plot_confusion_matrix, f1_score
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import TomekLinks
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Loading the data from csv file.
customer_df = pd.read_csv('/Users/pauli/Desktop/Ironhack/Week10/Jueves_29.09.2022/lab-feature-extraction/we_fn_use_c_marketing_customer_value_analysis.csv')
# Setting the display options.
pd.set_option('display.max_columns', None)
# Preview the dataset.
customer_df.head()

,Customer,State,Customer Lifetime Value,Response,Coverage,Education,Effective To Date,EmploymentStatus,Gender,Income,Location Code,Marital Status,Monthly Premium Auto,Months Since Last Claim,Months Since Policy Inception,Number of Open Complaints,Number of Policies,Policy Type,Policy,Renew Offer Type,Sales Channel,Total Claim Amount,Vehicle Class,Vehicle Size
0,BU79786,Washington,2763.519279,No,Basic,Bachelor,2/24/11,Employed,F,56274,Suburban,Married,69,32,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,QZ44356,Arizona,6979.535903,No,Extended,Bachelor,1/31/11,Unemployed,F,0,Suburban,Single,94,13,42,0,8,Personal Auto,Personal L3,Offer3,Agent,1131.464935,Four-Door Car,Medsize
2,AI49188,Nevada,12887.431650,No,Premium,Bachelor,2/19/11,Employed,F,48767,Suburban,Married,108,18,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
3,WW63253,California,7645.861827,No,Basic,Bachelor,1/20/11,Unemployed,M,0,Suburban,Married,106,18,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
4,HB64268,Washington,2813.692575,No,Basic,Bachelor,2/3/11,Employed,M,43836,Rural,Single,73,12,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize


In [3]:
customer_df.shape

(9134, 24)

In [4]:
# Drop redundant columns from the dataset which does not have any predictive power. 
# In this case, Customer is the redundant column. So, I will drop it first.
customer_df.drop('Customer', axis=1, inplace=True)

In [5]:
# Rename the columns so they follow the PE8 (snake case).
customer_df.columns = customer_df.columns.str.lower().str.replace(" ","_")
customer_df.rename(columns = {"employmentstatus" : "employment_status"}, inplace = True)
customer_df.columns

Index(['state', 'customer_lifetime_value', 'response', 'coverage', 'education',
       'effective_to_date', 'employment_status', 'gender', 'income',
       'location_code', 'marital_status', 'monthly_premium_auto',
       'months_since_last_claim', 'months_since_policy_inception',
       'number_of_open_complaints', 'number_of_policies', 'policy_type',
       'policy', 'renew_offer_type', 'sales_channel', 'total_claim_amount',
       'vehicle_class', 'vehicle_size'],
      dtype='object')

In [6]:
# Changing effective_to_date column to datetime format.
customer_df.effective_to_date = pd.to_datetime(customer_df.effective_to_date)
customer_df.effective_to_date.head()

0   2011-02-24
1   2011-01-31
2   2011-02-19
3   2011-01-20
4   2011-02-03
Name: effective_to_date, dtype: datetime64[ns]

### 1. We will start with removing outliers. 
So far, we have discussed different methods to remove outliers. Use the one you feel more comfortable with, define a function for that. Use the function to remove the outliers and apply it to the dataframe.

##### Taking the out outliers and plot remaining distribution.

* Outliers are extreme values below Q1 - 1.5 iqr or above Q3 + 1.5 iqr
* iqr = Q3 - Q1

In [7]:
def remove_outliers(data, col):
    Q3 = data[col].quantile(0.75)
    Q1 = data[col].quantile(0.25) 
    iqr = Q3 - Q1
    upper_limit = Q3 + 1.5 * iqr
    lower_limit = Q1 - 1.5 * iqr
    data_final=data[~((data[col] < lower_limit)|(data[col] > upper_limit))]
    data_final.reset_index(drop=True, inplace=True) 
    return data_final

customer_df= remove_outliers(customer_df, "total_claim_amount")
customer_df.head()

,state,customer_lifetime_value,response,coverage,education,effective_to_date,employment_status,gender,income,location_code,marital_status,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,number_of_open_complaints,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size
0,Washington,2763.519279,No,Basic,Bachelor,2011-02-24,Employed,F,56274,Suburban,Married,69,32,5,0,1,Corporate Auto,Corporate L3,Offer1,Agent,384.811147,Two-Door Car,Medsize
1,Nevada,12887.431650,No,Premium,Bachelor,2011-02-19,Employed,F,48767,Suburban,Married,108,18,38,0,2,Personal Auto,Personal L3,Offer1,Agent,566.472247,Two-Door Car,Medsize
2,California,7645.861827,No,Basic,Bachelor,2011-01-20,Unemployed,M,0,Suburban,Married,106,18,65,0,7,Corporate Auto,Corporate L2,Offer1,Call Center,529.881344,SUV,Medsize
3,Washington,2813.692575,No,Basic,Bachelor,2011-02-03,Employed,M,43836,Rural,Single,73,12,44,0,1,Personal Auto,Personal L1,Offer1,Agent,138.130879,Four-Door Car,Medsize
4,Oregon,8256.297800,Yes,Basic,Bachelor,2011-01-25,Employed,F,62902,Rural,Married,69,14,94,0,2,Personal Auto,Personal L3,Offer2,Web,159.383042,Two-Door Car,Medsize


In [8]:
customer_df.shape

(8681, 23)

### 2. Create a copy of the dataframe for the data wrangling.

In [9]:
df = customer_df.copy()

In [10]:
def diff_function(data):
        df_categoricals = pd.DataFrame()
        df_continuous = pd.DataFrame()
        df_discretes = pd.DataFrame()
        for i in data:  
            if np.dtype(data[i]) == 'object':
                df_categoricals[i] = data[i]
            elif len(data[i].unique()) > 20 or np.dtype(data[i]) == 'float':  #(len(data[i].unique())*0.8):
                df_continuous[i] = data[i]
            else: 
                df_discretes[i] = data[i]
        return df_categoricals, df_continuous, df_discretes
    
df_categoricals, df_continuous, df_discretes = diff_function(df)

### 3. Normalize the continuous variables. You can use any one method you want.

* Normalization is about transforming the feature values to fall within the bounded intervals (min and max).
* MinMaxScaler class of sklearn.preprocessing is used for normalization of features.

In [11]:
# We will only use the continuous numeric variables.
df_continuous.drop(['effective_to_date'] , axis = 1, inplace = True)
df_continuous.head()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,total_claim_amount
0,2763.519279,56274,69,32,5,384.811147
1,12887.431650,48767,108,18,38,566.472247
2,7645.861827,0,106,18,65,529.881344
3,2813.692575,43836,73,12,44,138.130879
4,8256.297800,62902,69,14,94,159.383042


In [12]:
# To normalize continuous features we apply the MinMaxScaler.
numerical_transformer = MinMaxScaler()
x_norm = numerical_transformer.fit_transform(df_continuous)

df_continuous = pd.DataFrame(x_norm, columns=df_continuous.columns)
df_continuous.head()

,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,total_claim_amount
0,0.013112,0.562847,0.034043,0.914286,0.050505,0.400735
1,0.166483,0.487763,0.200000,0.514286,0.383838,0.589962
2,0.087076,0.000000,0.191489,0.514286,0.656566,0.551847
3,0.013872,0.438443,0.051064,0.342857,0.444444,0.143781
4,0.096324,0.629140,0.034043,0.400000,0.949495,0.165918


### 4. Encode the categorical variables
#### Hint for Categorical Variables
You should deal with the categorical variables as shown below (for ordinal encoding, dummy code has been provided as well):
* One hot to state
* Ordinal to coverage
* Ordinal to employmentstatus
* Ordinal to location code
* One hot to marital status
* One hot to policy type
* One hot to policy
* One hot to renew offercustomer_df
* One hot to sales channel
* One hot vehicle class
* Ordinal vehicle size

data["coverage"] = data["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
* given that column "coverage" in the dataframe "data" has three categories:
* "basic", "extended", and "premium" and values are to be represented in the same order.

    Ordinal Encoding

In [13]:
# Define the features.
ordinal_list = ['coverage', 'employment_status', 'location_code', 'vehicle_size']
ordinal_encoding = df_categoricals.loc[:, ordinal_list]

In [14]:
for col in ordinal_encoding.columns:
    print(ordinal_encoding[col].value_counts())

Basic       5442
Extended    2556
Premium      683
Name: coverage, dtype: int64
Employed         5548
Unemployed       2066
Medical Leave     409
Disabled          383
Retired           275
Name: employment_status, dtype: int64
Suburban    5328
Rural       1773
Urban       1580
Name: location_code, dtype: int64
Medsize    6120
Small      1666
Large       895
Name: vehicle_size, dtype: int64


In [15]:
ordinal_encoding["coverage"] = ordinal_encoding["coverage"].map({"Basic" : 0, "Extended" : 1, "Premium" : 2})
ordinal_encoding["employment_status"] = ordinal_encoding["employment_status"].map({"Employed" : 0, "Unemployed" : 1, "Medical Leave" : 2, "Disabled" : 3, "Retired" : 4})
ordinal_encoding["location_code"] = ordinal_encoding["location_code"].map({"Rural" : 0, "Suburban" : 1, "Urban" : 2})
ordinal_encoding["vehicle_size"] = ordinal_encoding["vehicle_size"].map({"Small" : 0, "Medsize" : 1, "Large" : 2})

In [16]:
for col in ordinal_encoding.columns:
    print(ordinal_encoding[col].value_counts())

0    5442
1    2556
2     683
Name: coverage, dtype: int64
0    5548
1    2066
2     409
3     383
4     275
Name: employment_status, dtype: int64
1    5328
0    1773
2    1580
Name: location_code, dtype: int64
1    6120
0    1666
2     895
Name: vehicle_size, dtype: int64


    One-Hot Encoding

In [17]:
# Define the features.
one_hot_list = ['state', 'response', 'education', 'gender', 'marital_status', 'policy_type', 'policy', 'renew_offer_type', 'sales_channel', 'vehicle_class']
one_hot_encoding = df_categoricals.loc[:, one_hot_list]
one_hot_encoding.head()

,state,response,education,gender,marital_status,policy_type,policy,renew_offer_type,sales_channel,vehicle_class
0,Washington,No,Bachelor,F,Married,Corporate Auto,Corporate L3,Offer1,Agent,Two-Door Car
1,Nevada,No,Bachelor,F,Married,Personal Auto,Personal L3,Offer1,Agent,Two-Door Car
2,California,No,Bachelor,M,Married,Corporate Auto,Corporate L2,Offer1,Call Center,SUV
3,Washington,No,Bachelor,M,Single,Personal Auto,Personal L1,Offer1,Agent,Four-Door Car
4,Oregon,Yes,Bachelor,F,Married,Personal Auto,Personal L3,Offer2,Web,Two-Door Car


In [18]:
one_hot_encoding = pd.get_dummies(data= one_hot_encoding, columns = one_hot_list, drop_first=True)
one_hot_encoding.head()

,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Corporate L2,policy_Corporate L3,policy_Personal L1,policy_Personal L2,policy_Personal L3,policy_Special L1,policy_Special L2,policy_Special L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [19]:
df_categoricals = pd.concat([ordinal_encoding, one_hot_encoding], axis=1)
df_categoricals.head()

,coverage,employment_status,location_code,vehicle_size,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Corporate L2,policy_Corporate L3,policy_Personal L1,policy_Personal L2,policy_Personal L3,policy_Special L1,policy_Special L2,policy_Special L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
1,2,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
3,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1


In [20]:
df = pd.concat([df_categoricals, df_continuous, df_discretes], axis=1)
df.head()

,coverage,employment_status,location_code,vehicle_size,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Corporate L2,policy_Corporate L3,policy_Personal L1,policy_Personal L2,policy_Personal L3,policy_Special L1,policy_Special L2,policy_Special L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,total_claim_amount,number_of_open_complaints,number_of_policies
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.013112,0.562847,0.034043,0.914286,0.050505,0.400735,0,1
1,2,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.166483,0.487763,0.200000,0.514286,0.383838,0.589962,0,2
2,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0.087076,0.000000,0.191489,0.514286,0.656566,0.551847,0,7
3,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.013872,0.438443,0.051064,0.342857,0.444444,0.143781,0,1
4,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0.096324,0.629140,0.034043,0.400000,0.949495,0.165918,0,2


### 5. The time variable can be useful. Try to transform its data into a useful one. Hint: Day week and month as integers might be useful.

In [21]:
customer_df.effective_to_date.head()

0   2011-02-24
1   2011-02-19
2   2011-01-20
3   2011-02-03
4   2011-01-25
Name: effective_to_date, dtype: datetime64[ns]

In [22]:
df['day'] = customer_df.effective_to_date.dt.day
df['week'] = customer_df.effective_to_date.dt.week
df['month'] = customer_df.effective_to_date.dt.month
df['year'] = customer_df.effective_to_date.dt.year

### 6. Since the model will only accept numerical data, check and make sure that every column is numerical, if some are not, change it using encoding.

In [23]:
df.head()

,coverage,employment_status,location_code,vehicle_size,state_California,state_Nevada,state_Oregon,state_Washington,response_Yes,education_College,education_Doctor,education_High School or Below,education_Master,gender_M,marital_status_Married,marital_status_Single,policy_type_Personal Auto,policy_type_Special Auto,policy_Corporate L2,policy_Corporate L3,policy_Personal L1,policy_Personal L2,policy_Personal L3,policy_Special L1,policy_Special L2,policy_Special L3,renew_offer_type_Offer2,renew_offer_type_Offer3,renew_offer_type_Offer4,sales_channel_Branch,sales_channel_Call Center,sales_channel_Web,vehicle_class_Luxury Car,vehicle_class_Luxury SUV,vehicle_class_SUV,vehicle_class_Sports Car,vehicle_class_Two-Door Car,customer_lifetime_value,income,monthly_premium_auto,months_since_last_claim,months_since_policy_inception,total_claim_amount,number_of_open_complaints,number_of_policies,day,week,month,year
0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.013112,0.562847,0.034043,0.914286,0.050505,0.400735,0,1,24,8,2,2011
1,2,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.166483,0.487763,0.200000,0.514286,0.383838,0.589962,0,2,19,7,2,2011
2,0,1,1,1,1,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0.087076,0.000000,0.191489,0.514286,0.656566,0.551847,0,7,20,3,1,2011
3,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.013872,0.438443,0.051064,0.342857,0.444444,0.143781,0,1,3,5,2,2011
4,0,0,0,1,0,0,1,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0.096324,0.629140,0.034043,0.400000,0.949495,0.165918,0,2,25,4,1,2011


In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8681 entries, 0 to 8680
Data columns (total 49 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   coverage                        8681 non-null   int64  
 1   employment_status               8681 non-null   int64  
 2   location_code                   8681 non-null   int64  
 3   vehicle_size                    8681 non-null   int64  
 4   state_California                8681 non-null   uint8  
 5   state_Nevada                    8681 non-null   uint8  
 6   state_Oregon                    8681 non-null   uint8  
 7   state_Washington                8681 non-null   uint8  
 8   response_Yes                    8681 non-null   uint8  
 9   education_College               8681 non-null   uint8  
 10  education_Doctor                8681 non-null   uint8  
 11  education_High School or Below  8681 non-null   uint8  
 12  education_Master                86

    All the columns are numericals.